# 继承Module类构建模型

In [3]:
import torch
from torch import nn

class MLP(nn.Module):
    # 声明带有模型参数的层，这里声明的俩全连接层
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs) # 这里是如何进行初始化的？ 没大看懂
        self.hidden = nn.Linear(784, 256)
        self.act = nn.ReLU()
        self.output = nn.Linear(256, 10)
        
    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)

In [4]:
X = torch.rand(2, 784)
net = MLP()
print(net)
net(X)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.0227, -0.0161, -0.1010,  0.0576,  0.1119,  0.0336, -0.0562, -0.0737,
         -0.0137,  0.1680],
        [ 0.0704, -0.1022, -0.0475,  0.0187, -0.0193,  0.1004, -0.0960, -0.0112,
          0.0333,  0.1052]], grad_fn=<AddmmBackward0>)

# Sequential 类
当模型的前向计算为简单串联各个层的计算时，Sequential类可以通过更简单的方式定义模型

In [6]:
from collections import OrderedDict
class MySequential(nn.Module):
    def __init__(self, *args):
        super(MySequential, self).__init__()
        if len(args) == 1 and isinstance(args[0], OrderedDict): # 如果传入的是一个OrderedDict
            for key, module in args[0].items():
                self.add_module(key, module)
        else:
            for idx, module in enumerate(args): # enumerate 同时返回对象和其索引
                self.add_module(str(idx), module)
    def forward(self, input):
        for module in self._modules.values():
            input = module(input)
        return  input

In [7]:
net = MySequential(
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)
print(net)
net(X)

MySequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-0.3190,  0.0221, -0.2829,  0.0423,  0.1976, -0.0507, -0.1024,  0.1020,
         -0.1208, -0.3906],
        [-0.2063,  0.0157, -0.1608, -0.0568,  0.3853,  0.0317,  0.1061, -0.0616,
         -0.1134, -0.1416]], grad_fn=<AddmmBackward0>)

# ModuleList 类
接受一个子模块的列表作为输入，可以像List一样进行append和extend操作

In [8]:
net = nn.ModuleList(
    [nn.Linear(784, 256), nn.ReLU()]
)
net.append(nn.Linear(256, 10))
print(net[-1])
print(net)

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


# ModuleDict 类
接受一个子模块的字典作为收入，也可以类似字典进行添加访问操作

In [9]:
net = nn.ModuleDict(
    {
        'linear': nn.Linear(784, 256),
        'act': nn.ReLU(),
    }
)

net['output'] = nn.Linear(256, 10)
print(net['linear'])
print(net.output)
print(net)

Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)
